In [ ]:
from bertopic import BERTopic
import pickle as pkl
import pandas as pd
import numpy as np
import random
from bertopic.vectorizers import ClassTfidfTransformer

/Users/calvq/miniconda3/envs/10701-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# get original reviews
with open("../reviews.pkl", "rb") as file:
    reviews = pkl.load(file)
    print(reviews[0])

This was the first place in Vegas where the Yelpers let me down. The salt and pepper shrimp appetizer was inedible: all of the shell was left on underneath the crispy fried coating. As a result, the sauce used to marinate the shrimp never penetrated the meat, it coagulated in the head, and EXPLODED on you upon taking a bite. GROSS! The potstickers and the beef chow fun lacked any flavor whatsoever. The only redeeming quality (the service is pretty awful) is the roast duck: crispy and tasty skin with relatively moist meat and a relative bargain compared to places in NY and L.A.


In [8]:
%%python ../../NER/NER.py
from NER import NER

In [ ]:
import tqdm

ner = NER()
counter = 0
for review in tqdm.tqdm(reviews, leave=False):
    flag, output = ner.preprocess_text(review)
    if flag:
        counter += 1

print(counter)
print(len(reviews))

KeyboardInterrupt: 

In [5]:
# get the cleaned reviews 
with open("clean_reviews.pkl", "rb") as file:
    clean_reviews = pkl.load(file)
    print(clean_reviews[0])

['dr', 'goldberg', 'offer', 'everything', 'look', 'for', 'in', 'general', 'practitioner', 'he', 'nice', 'and', 'easy', 'to', 'talk', 'to', 'without', 'being', 'patronizing', 'he', 'always', 'on', 'time', 'in', 'seeing', 'his', 'patient', 'he', 'affiliated', 'with', 'top', 'notch', 'hospital', 'nyu', 'which', 'my', 'parent', 'have', 'explained', 'to', 'me', 'is', 'very', 'important', 'in', 'case', 'something', 'happens', 'and', 'you', 'need', 'surgery', 'and', 'you', 'can', 'get', 'referral', 'to', 'see', 'specialist', 'without', 'having', 'to', 'see', 'him', 'first', 'really', 'what', 'more', 'do', 'you', 'need', 'sitting', 'here', 'trying', 'to', 'think', 'of', 'any', 'complaint', 'have', 'about', 'him', 'but', 'really', 'drawing', 'blank']


In [6]:
print(len(clean_reviews))
# index_list = range(len(clean_reviews))
# random.sample(index_list, 10000)
random.seed(10701)
clean_reviews_down = random.sample(clean_reviews, 10000)
print(len(clean_reviews_down))

700000
10000


In [8]:
# get data in string type
processed_corpus = [" ".join(tokens) for tokens in clean_reviews_down]

In [9]:
# method 1: guided topic modeling

seed_topic_list = [
    ["pizza", "food", "chicken", "burger", "taco", "salsa", "mexican", "chip", "bean", "burrito", "enchilada", "rice", "tortilla", "guacamole", "carne", "asada", "shrimp", "lobster", "sushi", "roll", "fish", "sashimi", "tuna", "tempura"],
    ["doctor", "office", "medical", "service", "patient", "health", "insurance", "client", "car", "vehicle"], 
    ["hair", "nail", "salon", "gel", "polish", "manicure"], #maybe counts as service? 
    "de le la u00e9 et est u00e0 un com pa www que da en die yelp u00e9e du select une pour service au mais plus".split(" "), #french
    "bar drink beer night bartender table music atmosphere".split(" "), #bars
    "hotel bed bath shower spa pool casino strip night desk check club show sexy men girl".split(" "),#vegas
    "store shop price item shopping buy need sale mall product boutique outlet walmart".split(" "),#shopping malls
]


topic_model = BERTopic(seed_topic_list=seed_topic_list, embedding_model="all-MiniLM-L6-v2")
topics, probs = topic_model.fit_transform(processed_corpus)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [11]:
topic_model.get_topic(0)

[('hotel', 0.02655898844111409),
 ('room', 0.02601600934295312),
 ('casino', 0.011447271056643988),
 ('stay', 0.011243704347691839),
 ('the', 0.01119048736400992),
 ('pool', 0.010993659611008044),
 ('bed', 0.010445518928920877),
 ('in', 0.00980066241962438),
 ('to', 0.009528182724897428),
 ('at', 0.008842415827258124)]

In [13]:
topic_model.visualize_topics()


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[0, 'hotel | room | casino | stay | the', 524],
                                   [1, 'taco | mexican | salsa | chip | burrito', 493],
                                   [2, 'vega | buffet | food | the | in', 297],
                                   ...,
                                   [93, 'noodle | rice | chicken | soup | sauce', 12],
                                   [94, 'patio | atmosphere | great | men | wonderful', 10],
                                   [95, 'audi | car | truck | serviced | brake', 10]], dtype=object),
              'hovertemplate': '<b>Topic %{customdata[0]}</b><br>%{customdata[1]}<br>Size: %{customdata[2]}',
              'legendgroup': '',
              'marker': {'color': '#B0BEC5',
                         'line': {'color': 'DarkSlateGrey', 'width': 2},
                         'size': array([524, 493, 297, 267, 197, 176, 170, 132, 130, 118, 112, 109, 107, 107,
                                        107, 102, 101,  99,  96,  91,  81,  81,  79,  73,  71,  71,  70,  62,
                                         57,  55,  55,  53,  52,  51,  47,  46,  44,  44,  43,  41,  39,  39,
                                         36,  35,  34,  34,  32,  32,  31,  30,  29,  29,  29,  27,  26,  25,
                                         25,  24,  24,  23,  23,  22,  22,  22,  22,  22,  20,  20,  20,  19,
                                         19,  19,  19,  18,  18,  18,  18,  18,  17,  16,  16,  16,  16,  15,
                                         15,  15,  14,  14,  14,  13,  13,  13,  12,  12,  10,  10]),
                         'sizemode': 'area',
                         'sizeref': 0.3275,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array([15.0025015 , -2.8943536 , -3.7913864 ,  0.64620763,  8.534175  ,
                          -3.5133405 , 19.124687  ,  4.2150974 , -3.1839166 , -3.6574945 ,
                          -0.08585793, -3.1461687 , -3.8617532 ,  7.5293207 , 15.100112  ,
                          19.275103  ,  0.45301226, -7.6079044 ,  8.788667  , -8.16482   ,
                           8.977365  ,  0.35266674,  0.6929511 ,  8.977722  ,  8.159174  ,
                          -3.8810701 , -0.4146481 ,  4.26938   ,  9.868979  ,  8.220436  ,
                          19.284351  ,  7.684309  ,  8.129725  , -8.141392  , -4.241518  ,
                          -2.9763658 , -0.7171806 ,  8.629899  ,  4.179617  , 19.396147  ,
                          18.98738   ,  9.828208  ,  8.129588  , -3.2561283 ,  9.334867  ,
                          15.147115  , -0.66885006,  4.3923697 ,  1.4902669 , -4.27613   ,
                           8.466224  ,  7.649469  ,  8.067822  ,  0.17512403, -8.237273  ,
                          -4.0241895 , -3.864608  , -0.30631402, -3.3039153 ,  8.363458  ,
                          -7.879541  , -3.794227  , -3.8693182 ,  9.43533   ,  8.826977  ,
                          -7.926702  , -3.199261  , -0.33654183, -0.12872748,  4.3141947 ,
                           9.63725   , -3.0730543 ,  4.299845  ,  4.000299  , -4.3287573 ,
                           1.4904509 ,  4.16384   , -7.4323506 , -7.776373  , -2.9790995 ,
                          -3.6256866 ,  0.7884547 ,  4.086389  ,  9.740828  ,  4.1362104 ,
                          15.199581  , -2.7333176 , -3.1201773 ,  8.19916   , -3.0306365 ,
                          -4.369364  , -2.6864064 ,  4.1671042 , -7.878181  , -3.014661  ,
                           7.8781137 ], dtype=float32),
              'xaxis': 'x',
              'y': array([  2.7020335 ,   4.8406687 ,   4.1671205 ,  -7.2980394 ,   8.023286  ,
                            5.9802322 ,  -2.1734746 ,   0.3515594 ,  22.103596  ,   5.849829  ,
                           -6.386901  ,  22.065807  ,   5.610602  , -12.12316   ,   2.8015592 

In [19]:
# method 2: seeded words 
# https://maartengr.github.io/BERTopic/getting_started/seed_words/seed_words.html

# we can also specify the number of topics using a customized UMAP model
# umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')

ctfidf_model = ClassTfidfTransformer(
    seed_words=["pizza", "doctor", "salon", "casino", "car"], 
    seed_multiplier=2
)

# We run the topic model with the seeded words
seed_topic_model = BERTopic(
    min_topic_size=10,
    ctfidf_model=ctfidf_model,
).fit(processed_corpus)


In [20]:
seed_topic_model.get_topic(0)

[('taco', 0.022344823668546638),
 ('mexican', 0.018764607428293764),
 ('salsa', 0.016875962108340734),
 ('chip', 0.011354993455712323),
 ('burrito', 0.011107642597563498),
 ('food', 0.009275184680470875),
 ('the', 0.009235481312870844),
 ('bean', 0.009219418105274624),
 ('and', 0.009191142693618586),
 ('good', 0.009116329264203183)]

In [21]:
seed_topic_model.visualize_topics()


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[0, 'taco | mexican | salsa | chip | burrito', 478],
                                   [1, 'room | hotel | casino | stay | the', 437],
                                   [2, 'vega | buffet | the | food | in', 296],
                                   ...,
                                   [101, 'salad | beet | ngreat | kid | barbacoa', 10],
                                   [102, 'charlotte | checker | play | uptown | raleigh', 10],
                                   [103, 'lunch | falafel | sandwich | wildflower | menu', 10]],
                                  dtype=object),
              'hovertemplate': '<b>Topic %{customdata[0]}</b><br>%{customdata[1]}<br>Size: %{customdata[2]}',
              'legendgroup': '',
              'marker': {'color': '#B0BEC5',
                         'line': {'color': 'DarkSlateGrey', 'width': 2},
                         'size': array([478, 437, 296, 265, 248, 209, 192, 161, 117, 115, 113, 108, 102, 101,
                                        101,  87,  84,  84,  83,  79,  73,  72,  71,  70,  69,  69,  55,  54,
                                         50,  49,  49,  48,  48,  47,  45,  43,  40,  39,  38,  36,  35,  34,
                                         34,  33,  32,  31,  31,  29,  29,  26,  26,  26,  26,  26,  25,  25,
                                         25,  25,  24,  24,  24,  24,  23,  23,  23,  23,  23,  23,  22,  22,
                                         22,  22,  22,  21,  21,  19,  19,  19,  19,  18,  18,  17,  17,  17,
                                         16,  15,  15,  14,  14,  14,  13,  13,  13,  13,  12,  12,  12,  12,
                                         12,  11,  10,  10,  10,  10]),
                         'sizemode': 'area',
                         'sizeref': 0.29875,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array([ -3.4268596 ,  -0.7806151 ,  -3.5423248 ,   1.5788188 ,  -3.8023274 ,
                            6.2229166 ,  -2.092401  ,  -2.3269951 ,  10.499567  ,  -9.833565  ,
                            7.8949056 ,   2.7468297 ,  15.168724  ,  -9.504407  ,   1.6103218 ,
                            1.2331339 ,  -3.7855368 ,   2.7196782 ,  15.152458  ,   7.858144  ,
                           14.994904  ,  14.546129  ,  -4.3780246 ,   1.2021706 ,  25.628448  ,
                            5.323503  ,  -2.2566674 ,   7.1117926 ,  -3.8377752 ,   5.2950287 ,
                           -9.414564  ,  -3.3798513 ,  15.071334  ,   3.3035204 ,   6.138009  ,
                           12.808465  ,   7.478439  ,  25.69148   ,   7.403601  ,  -1.8768954 ,
                           12.55136   ,  -9.996659  ,  -3.12749   ,  -3.0359693 ,  -3.543779  ,
                            6.5816655 ,   7.269863  ,  -2.706153  ,  -3.2562366 ,   6.5419283 ,
                           12.622361  ,  -9.378328  ,  -2.9909441 ,   7.0426536 , -10.100311  ,
                           -2.16916   ,   6.320641  ,   5.2231445 ,  -1.9274336 ,  25.808884  ,
                            8.037622  ,   7.360458  ,   1.7979352 ,   8.046837  ,  14.505778  ,
                           12.726157  ,   2.8478827 ,  -2.3154726 ,  -3.5936687 ,  15.683173  ,
                            2.9436603 ,  -4.696226  ,  15.406818  ,   6.5820265 ,  -9.80038   ,
                           10.375762  ,  25.60477   ,  -3.367989  ,  25.583181  ,   7.0306287 ,
                           15.439678  ,  -4.329483  ,   6.9820757 ,  -2.2089682 ,   1.7671673 ,
                           12.504205  ,  -0.78061944,   2.7060661 ,  12.759945  , -10.154161  ,
                            6.832865  ,  25.504414  ,  -2.1723201 ,  -4.655735  ,  15.5987015 ,
                            3.303557  ,  10.14601   ,   1.122158  ,   2.6988077 ,  10.242311  ,
                           -3.8865912 ,  -2.7541974